<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AOC_MULTIPLE_LLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai -q
!pip install anthropic -q
!pip install colab-env -q

In [3]:
from IPython import get_ipython
from IPython.display import display

import os
from typing import Dict, Any, List, Optional
import google.generativeai as genai
from anthropic import Anthropic
import colab_env # For Google Colab

# Import necessary libraries - remains the same for core API clients
try:
    import google.generativeai as genai
    from anthropic import Anthropic
    from anthropic.types import MessageParam
    # Conditional import for Colab's userdata
    try:
        from google.colab import userdata
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False
        print("Running outside Google Colab. 'userdata.get' will not be available.")

except ImportError:
    print("\n[CRITICAL ERROR] Please install necessary libraries:")
    print("pip install google-generativeai anthropic")
    print("Exiting as API functionality cannot be demonstrated without them.")
    # In a real application, you might raise an exception or handle this more gracefully
    # For this example, we'll allow execution but API calls will be mocked
    pass


# --- 1. Core Components (Interfaces and Implementations with API Management) ---

class BaseModel:
    """Base class for Language Models and Reasoning Models."""
    def __init__(self, name: str, identifier: str, api_client: Any):
        self.name = name
        self.identifier = identifier # The specific model string
        self.api_client = api_client # The actual API client instance

    def generate(self, prompt: str, **kwargs) -> str:
        raise NotImplementedError("Subclasses must implement 'generate'")

class OperationsLanguageModel(BaseModel):
    """Integrates with a general LLM API (e.g., Gemini) for broad operational tasks."""
    def __init__(self, name: str, identifier: str, api_client: Optional[genai.GenerativeModel]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, genai.GenerativeModel):
             print(f"Warning: Expected genai.GenerativeModel for {identifier}, but received {type(api_client)}. Mocking API calls.")
             self.api_client = None

    def generate(self, prompt: str, temperature: float = 0.5, max_tokens: int = 400) -> str: # Slightly lower temp/tokens for operational focus
        """
        Makes an actual API call or uses mock response.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to generate operational response for prompt: '{prompt[:80]}...'")

        if self.api_client:
            try:
                generation_config = {
                    "temperature": temperature,
                    "max_output_tokens": max_tokens,
                }
                response = self.api_client.generate_content(
                    prompt,
                    generation_config=generation_config
                )
                if response and hasattr(response, 'text') and response.text:
                    return response.text
                elif response and hasattr(response, 'prompt_feedback'):
                    print(f"Prompt feedback from OLM: {response.prompt_feedback}")
                    return f"API did not generate content due to safety concerns or other issues."
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling OLM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Generating operational response for prompt: '{prompt[:80]}...'")
        if "flight 123 status" in prompt.lower():
            return f"[{self.name}] (MOCK) Analyzing status for Flight 123. Key findings: Departed on time, currently en route. Need to check weather ahead and destination gate."
        elif "weather at jfk" in prompt.lower():
             return f"[{self.name}] (MOCK) Checking weather at JFK. Conditions reported as cloudy with moderate winds. Need to retrieve latest METAR/TAF."
        return f"[{self.name}] (MOCK) Generated a general operational response based on: '{prompt}'"


class OperationalReasoningModel(BaseModel):
    """Integrates with a specialized reasoning API (e.g., Claude Sonnet) for complex operational scenarios."""
    def __init__(self, name: str, identifier: str, api_client: Optional[Anthropic]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, Anthropic):
             print(f"Warning: Expected anthropic.Anthropic for {identifier}, but received {type(api_client)}. Mocking API calls.")
             self.api_client = None


    def generate(self, prompt: str, complexity_level: str = "high") -> str:
        """
        Makes an actual API call or uses mock response.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to perform operational reasoning for prompt: '{prompt[:80]}...' (Complexity: {complexity_level})")

        if self.api_client:
            try:
                messages: List[MessageParam] = [
                    {"role": "user", "content": prompt}
                ]
                response = self.api_client.messages.create(
                    model=self.identifier,
                    max_tokens=1000,
                    messages=messages,
                    temperature=0.1, # Low temperature for deterministic reasoning
                )
                if response and hasattr(response, 'content') and response.content:
                    return "".join(block.text for block in response.content if hasattr(block, 'text'))
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling ORM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Performing operational reasoning for prompt: '{prompt[:80]}...'")
        if "reroute flight 456 due to weather" in prompt.lower():
            return f"[{self.name}] (MOCK) ORM reasoned: Evaluated weather diversion options for flight 456. Suggested alternate airports include ORD and DTW based on forecast. Need to confirm fuel, payload, and ETOPS requirements. Check NOTAMs."
        elif "evaluate crew duty time for flight 789" in prompt.lower():
            return f"[{self.name}] (MOCK) ORM reasoned: Calculated crew duty times for flight 789 based on Part 117 regulations. Crew appears legal for planned flight. Monitor for potential delays impacting duty limits."
        return f"[{self.name}] (MOCK) ORM performed complex operational reasoning based on: '{prompt}'"

# --- Mock Retrieval Tools (Adapted for Airline Operations) ---

class OperationalMemory:
    """Manages short-term and long-term operational memory."""
    def __init__(self):
        self.short_term_memory: List[str] = []
        self.long_term_memory: List[str] = [] # Could be a more complex data structure

    def add_short_term(self, entry: str):
        self.short_term_memory.append(entry)
        # Keep short-term memory limited
        if len(self.short_term_memory) > 15: # Slightly more short-term memory for operational context
            self.short_term_memory.pop(0)

    def add_long_term(self, entry: str):
        self.long_term_memory.append(entry)
        # In a real system, this might involve embeddings and indexing for flight data, regs, etc.

    def retrieve_short_term(self, query: str = "") -> List[str]:
        # Simple retrieval; in real system, would be more sophisticated (e.g., semantic search)
        return [m for m in self.short_term_memory if query.lower() in m.lower()]

    def retrieve_long_term(self, query: str = "") -> List[str]:
        return [m for m in self.long_term_memory if query.lower() in m.lower()]

class FlightDataDatabase:
    """Mock Database for Flight Information."""
    def query(self, flight_identifier: str, data_type: str = "status") -> List[str]:
        print(f"Querying Flight Data DB for: '{flight_identifier}', data type: '{data_type}'...")
        # Simulate results
        if flight_identifier == "FL123":
            if data_type == "status":
                return ["FL123: Status: Airborne", "FL123: Departure: LAX (10:00 PST)", "FL123: Destination: JFK (18:30 EST)"]
            elif data_type == "details":
                 return ["FL123: Aircraft: B737", "FL123: Crew: Smith, Jones", "FL123: Route: LAX-JFK via J77"]
        return [f"Flight data for {flight_identifier} not found in mock DB."]

class OperationalKnowledgeBase:
    """Mock Knowledge Base for Regulations, Manuals, etc."""
    def query(self, natural_language_query: str) -> List[str]:
        print(f"Querying Operational Knowledge Base for: '{natural_language_query}'...")
        # Simulate results related to regulations or procedures
        if "part 121 regulations" in natural_language_query or "duty limits" in natural_language_query:
            # Reference to relevant regulations [2]
            return ["FAR Part 121 (Domestic/Flag/Supplemental)", "FAR Part 117 (Flightcrew Duty/Rest)", "Reference: 14 CFR § 135.619 - Operations control centers. [1]"]
        elif "de-icing procedures" in natural_language_query:
             return ["Aircraft De-icing Procedure (Manual PQR)", "Holdover Time (HOT) charts link."]
        return ["Operational concept XYZ from knowledge base.", "Procedure ABC found in manual."]

class ExternalWeatherService:
    """Mock Service for Weather Information (METAR, TAF, Forecasts)."""
    def search(self, location_code: str, weather_type: str = "current") -> List[str]:
        print(f"Searching External Weather Service for: '{location_code}', type: '{weather_type}'...")
        if location_code.upper() == "JFK":
            if weather_type == "current":
                return ["JFK METAR: KUFK 261852Z 18010KT 10SM CLR 15/05 A3000...", "JFK TAF: TAF KUFK 261720Z 2618/2718 18010KT P6SM SCT040..."]
            elif weather_type == "forecast":
                 return ["JFK Area Forecast: ...showers expected after 22Z..."]
        elif location_code.upper() == "LAX":
             return ["LAX METAR: KLAX 261850Z 25005KT 10SM FEW250 20/10 A2998..."]
        return [f"Weather data for {location_code} not found in mock service."]

# --- 2. Retrieval Agents (Adapted for Airline Operations) ---

# --- 2a. Retrieval Agents ---

class RetrievalAgent:
    """Base class for retrieval agents."""
    def __init__(self, name: str):
        self.name = name

    def retrieve(self, query: str, **kwargs) -> List[str]:
        raise NotImplementedError("Subclasses must implement 'retrieve'")

class OperationalKnowledgeAgent(RetrievalAgent):
    """Retrieves from Flight Data DB and Operational Knowledge Base."""
    def __init__(self, flight_db: FlightDataDatabase, op_kb: OperationalKnowledgeBase):
        super().__init__("OperationalKnowledgeAgent")
        self.flight_db = flight_db
        self.op_kb = op_kb

    def retrieve(self, query: str, query_type: str = "knowledge") -> List[str]:
        results = []
        if query_type == "flight_data":
            # Assume query contains flight identifier for this type
            flight_id = query.split()[1] if len(query.split()) > 1 else "UNKNOWN"
            results.extend(self.flight_db.query(flight_id))
        elif query_type == "knowledge":
            results.extend(self.op_kb.query(query))
        return results

class OperationalSearchAgent(RetrievalAgent):
    """Retrieves from external weather services and potentially other operational sources."""
    def __init__(self, weather_service: ExternalWeatherService):
        super().__init__("OperationalSearchAgent")
        self.weather_service = weather_service
        # Could add more external services here

    def retrieve(self, query: str) -> List[str]:
        all_results = []
        # Basic logic to check for weather query
        if "weather at" in query.lower() or "metar" in query.lower() or "taf" in query.lower():
            parts = query.lower().split("weather at")
            location_code = parts[-1].strip().split()[0] if len(parts) > 1 else ""
            if location_code:
                all_results.extend(self.weather_service.search(location_code))
        # Add more external search types here as needed
        return all_results


# --- 3. Agentic Reasoning and Console (Adapted for Airline Operations) ---

class AgenticOperationalReasoning:
    """Orchestrates OLM, ORM, and Retrieval Agents for Airline Operational Tasks."""
    def __init__(self, olm_general: OperationsLanguageModel,
                 orm_operational: OperationalReasoningModel, memory: OperationalMemory,
                 kb_agent: OperationalKnowledgeAgent, se_agent: OperationalSearchAgent):
        self.olm_general = olm_general
        self.orm_operational = orm_operational
        self.memory = memory
        self.kb_agent = kb_agent
        self.se_agent = se_agent
        # Use the general OLM as the primary interpreter
        self.current_model = self.olm_general

    def process_query(self, modified_input: str) -> str:
        """
        This method embodies the core Agentic Reasoning loop for operational queries.
        """
        print(f"\n[Agentic Operational Reasoning] Processing input using {self.current_model.identifier} and {self.orm_operational.identifier}: '{modified_input[:80]}...'")
        self.memory.add_short_term(f"User query: {modified_input}")

        # Step 1: Initial OLM interpretation and plan using the general OLM
        olm_system_prompt_initial = "You are an AI airline operations assistant. Analyze the user query to determine information needs and initial steps for operational tasks. Prioritize retrieving relevant flight data, weather, and regulations."
        initial_olm_thought = self.current_model.generate(f"{olm_system_prompt_initial}\nUser query: {modified_input}")
        self.memory.add_short_term(f"OLM initial thought: {initial_olm_thought}")
        print(f"OLM Initial Thought: {initial_olm_thought}")

        # Step 2: Determine retrieval needs based on OLM's thought (Operational Keywords)
        retrieval_query = ""
        # Keywords updated for airline operations
        operational_keywords = ["flight status", "weather", "delay", "reroute", "diversion", "crew duty", "regulations", "manuals", "airport", "NOTAMs"]
        if any(keyword in initial_olm_thought.lower() for keyword in operational_keywords) or any(keyword in modified_input.lower() for keyword in operational_keywords):
             retrieval_query = modified_input # Use original query or refine it
             print(f"Determined retrieval need for operational information based on: '{retrieval_query[:80]}'")

             # Perform Retrieval using Operational Agents
             kb_results = self.kb_agent.retrieve(retrieval_query, query_type="knowledge") # Assume knowledge for most initial queries
             # Attempt to extract flight ID for specific flight data query
             flight_id_match = next((word for word in retrieval_query.split() if word.startswith('FL')), None)
             if flight_id_match:
                  kb_results.extend(self.kb_agent.retrieve(f"Get flight_data for {flight_id_match}", query_type="flight_data"))

             se_results = self.se_agent.retrieve(retrieval_query)
             retrieved_info = "\n".join(kb_results + se_results)
             print(f"Retrieved Operational Information: {retrieved_info[:200]}...")
             self.memory.add_short_term(f"Retrieved: {retrieved_info}")
        else:
             retrieved_info = "No specific operational retrieval needed."
             print(retrieved_info)


        # Step 3: Engage ORM (Operational Reasoning Model) if complex reasoning is required
        orm_system_prompt = "You are a specialized airline operational reasoning engine (e.g., Claude Sonnet). Analyze the gathered operational information and the user's original intent to formulate a precise analysis or recommendation (e.g., reroute options, duty time compliance, operational impacts). Focus on accuracy, regulatory compliance, and operational feasibility."
        # Keywords updated for operational reasoning tasks
        reasoning_keywords = ["analyze", "interpret", "evaluate", "suggest alternate", "recommend action", "compare routes", "predict impact", "manage situation"]
        if any(keyword in modified_input.lower() for keyword in reasoning_keywords) or any(keyword in initial_olm_thought.lower() for keyword in reasoning_keywords):
             reasoning_input = f"User query: {modified_input}\nInitial OLM thought: {initial_olm_thought}\nRetrieved info: {retrieved_info}"
             orm_output = self.orm_operational.generate(f"{orm_system_prompt}\n{reasoning_input}")
             self.memory.add_short_term(f"ORM operational reasoning: {orm_output}")
             print(f"ORM Operational Output: {orm_output}")
        else:
             orm_output = "No specific complex operational reasoning performed by the specialized model."
             print(orm_output)


        # Step 4: Final OLM Synthesis (General OLM)
        final_olm_prompt = f"Based on the user's original operational query: '{modified_input}', initial analysis: '{initial_olm_thought}', retrieved operational information: '{retrieved_info}', and specialized operational reasoning by {self.orm_operational.identifier}: '{orm_output}', synthesize a comprehensive, accurate, and professional operational response using {self.current_model.name}. Remember to state this is for informational purposes and decisions must comply with regulations and company procedures."
        final_response = self.current_model.generate(final_olm_prompt, max_tokens=800) # Increased tokens for final response
        self.memory.add_long_term(f"Interaction: Query='{modified_input}', Response='{final_response}'")
        print(f"\n[Final Operational Response] {final_response}")
        return final_response

class AgenticOperationalConsole:
    """The top-level console orchestrating user interaction for Airline Operational Queries."""
    def __init__(self, gemini_client: Optional[genai.GenerativeModel], claude_client: Optional[Anthropic]):
        # Initialize general LLM
        self.olm_general = OperationsLanguageModel("Gemini 2.0 Flash (General)", 'gemini-2.0-flash', gemini_client)
        # Initialize Financial Reasoning Model
        self.orm_operational = OperationalReasoningModel("Claude Opus 4 (Financial Reasoning)", 'claude-opus-4-20250514', claude_client)

        self.memory = OperationalMemory() # Use adapted memory class
        self.flight_db = FlightDataDatabase() # Mock Flight DB
        self.op_kb = OperationalKnowledgeBase() # Mock Operational Knowledge Base
        self.weather_service = ExternalWeatherService() # Mock Weather Service
        # Use Operational-specific Retrieval Agents
        self.kb_agent = OperationalKnowledgeAgent(self.flight_db, self.op_kb)
        self.se_agent = OperationalSearchAgent(self.weather_service)

        # Use AgenticOperationalReasoning
        self.agentic_reasoning = AgenticOperationalReasoning(
            olm_general=self.olm_general,
            orm_operational=self.orm_operational,
            memory=self.memory,
            kb_agent=self.kb_agent,
            se_agent=self.se_agent
        )

    def run_query(self, user_query: str) -> str:
        print(f"\n--- [Agentic Operational Console] Processing User Query: '{user_query}' ---")
        parsed_query = user_query.strip()
        print(f"Query Parsed: '{parsed_query}'")

        # System Prompt for the overall operational task
        system_prompt = "You are a helpful AI airline operations assistant. Your goal is to provide accurate and comprehensive analysis and information based on the provided operational data and retrieved knowledge. Always state this is for informational purposes and decisions must comply with regulations and company procedures."
        modified_input = f"{system_prompt}\nUser: {parsed_query}"

        # Agentic Reasoning for Operational Query
        compiled_output = self.agentic_reasoning.process_query(modified_input)

        print(f"--- [Agentic Operational Console] Query Complete ---")
        return compiled_output

# --- Example Usage (Adapted for Operational Query) ---
if __name__ == "__main__":
    print("Setting up API clients...")
    # --- Gemini API Setup ---
    GOOGLE_API_KEY = None
    if IN_COLAB:
        try:
            GOOGLE_API_KEY = userdata.get('GEMINI')
            print("Successfully retrieved GEMINI_API_KEY from Colab environment variable.")
        except Exception as e:
            print(f"Could not retrieve GEMINI_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not GOOGLE_API_KEY:
        GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY") # Fallback to environment variable
        if GOOGLE_API_KEY:
            print("Successfully retrieved GOOGLE_API_KEY from environment variable.")
        else:
            print("WARNING: GOOGLE_API_KEY not found. Gemini API calls will be mocked.")

    gemini_client = None
    if GOOGLE_API_KEY:
        try:
            genai.configure(api_key=GOOGLE_API_KEY)
            gemini_client = genai.GenerativeModel('gemini-2.0-flash')
            print("Gemini GenerativeModel ('gemini-2.0-flash') client initialized.")
        except Exception as e:
            print(f"ERROR initializing Gemini client: {e}. Gemini API calls will be mocked.")
    else:
        print("No Google API Key found, Gemini client will not be initialized.")

    # --- Claude API Setup ---
    CLAUDE_API_KEY = None
    if IN_COLAB:
        try:
            CLAUDE_API_KEY = os.environ.get("CLAUDE3_API_KEY")
            print("Successfully retrieved CLAUDE3_API_KEY from Colab userdata.")
        except Exception as e:
            print(f"Could not retrieve CLAUDE3_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not CLAUDE_API_KEY:
        CLAUDE_API_KEY = os.environ.get("ANTHROPIC_API_KEY") # Fallback to environment variable
        if CLAUDE_API_KEY:
            print("Successfully retrieved ANTHROPIC_API_KEY from environment variable.")
        else:
            print("WARNING: ANTHROPIC_API_KEY not found. Claude API calls will be mocked.")

    claude_client = None
    if CLAUDE_API_KEY:
        try:
            claude_client = Anthropic(api_key=CLAUDE_API_KEY)
            print("Anthropic Claude client initialized.")
        except Exception as e:
            print(f"ERROR initializing Anthropic client: {e}. Claude API calls will be mocked.")
    else:
        print("No Anthropic API Key found, Claude client will not be initialized.")


    # Instantiate the console with the prepared API clients
    console = AgenticOperationalConsole(gemini_client=gemini_client, claude_client=claude_client)
    print('\n')
    print("\nAgenticOperationalConsole initialized. Running your specific operational query...")
    print('\n')

    # --- Running Your Specific Operational Query ---
    # Example operational query
    your_operational_query = "Analyze the status of flight FL123 and the weather forecast for JFK to assess potential delays."

    print(f"\n--- Running Operational Query: {your_operational_query} ---")
    operational_output = console.run_query(your_operational_query)
    print(f"\n--- Operational Analysis Output ---")
    print(operational_output)
    print("--- End of Operational Analysis Output ---")

    # --- Displaying Memory Contents after your query ---
    print("\n--- Current Short-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_short_term())
    print('\n')

    print("\n--- Current Long-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_long_term())
    print('\n')

Setting up API clients...
Successfully retrieved GEMINI_API_KEY from Colab environment variable.
Gemini GenerativeModel ('gemini-2.0-flash') client initialized.
Successfully retrieved CLAUDE3_API_KEY from Colab userdata.
Anthropic Claude client initialized.



AgenticOperationalConsole initialized. Running your specific operational query...



--- Running Operational Query: Analyze the status of flight FL123 and the weather forecast for JFK to assess potential delays. ---

--- [Agentic Operational Console] Processing User Query: 'Analyze the status of flight FL123 and the weather forecast for JFK to assess potential delays.' ---
Query Parsed: 'Analyze the status of flight FL123 and the weather forecast for JFK to assess potential delays.'

[Agentic Operational Reasoning] Processing input using gemini-2.0-flash and claude-opus-4-20250514: 'You are a helpful AI airline operations assistant. Your goal is to provide accur...'
[Gemini 2.0 Flash (General) (gemini-2.0-flash)] Attempting to ge